# Testing package and different types of problems

In [1]:
using Pkg
Pkg.add("NonlinearSolve")

   Resolving package versions...
     Project No packages added to or removed from `~/.julia/environments/v1.12/Project.toml`
    Manifest No packages added to or removed from `~/.julia/environments/v1.12/Manifest.toml`


In [2]:
# general
using NonlinearSolve

f(u, p) = u .* u .- p
u0 = [1.0, 1.0]
p = 0
prob = NonlinearProblem(f, u0, p)
sol = solve(prob, NewtonRaphson())

retcode: Success
u: 2-element Vector{Float64}:
 4.76837158203125e-7
 4.76837158203125e-7

In [3]:
using BenchmarkTools

@benchmark solve(prob, NewtonRaphson())

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  10.375 μs … 524.125 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.209 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.884 μs ±   8.459 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▆██▇▆▆▅▄▃▂▁▁▁▂▂ ▁                                           ▂
  ███████████████████▇▆▆▆▆▆▆▆▃▄▃▄▄▆▃▁▅▃▃▄▃▁▄▄▄▃▃▅▁▃▁▅▄▃▆▃▅▄▄▁▃ █
  10.4 μs       Histogram: log(frequency) by time        24 μs <

 Memory estimate: 11.33 KiB, allocs estimate: 210.

In [4]:
Pkg.add("SteadyStateDiffEq")

   Resolving package versions...
     Project No packages added to or removed from `~/.julia/environments/v1.12/Project.toml`
    Manifest No packages added to or removed from `~/.julia/environments/v1.12/Manifest.toml`


In [5]:
# steady state problem
using NonlinearSolve, SteadyStateDiffEq

f(u, p, t) = [2 - 2u[1]; u[1] - 4u[2]]
u0 = [0.0, 0.0]
prob = SteadyStateProblem(f, u0)

solve(prob, SSRootfind())

┌ Warning: The `alias_u0` keyword argument is deprecated. Please use a NonlinearAliasSpecifier, e.g. `alias = NonlinearAliasSpecifier(alias_u0 = true)`.
└ @ NonlinearSolveBase ~/.julia/packages/NonlinearSolveBase/2E600/src/solve.jl:57


retcode: Success
u: 2-element Vector{Float64}:
 1.0
 0.25

In [6]:
# nonlinear least squares
using NonlinearSolve

function nlls!(du, u, p)
    du[1] = 2u[1] - 2
    du[2] = u[1] - 4u[2]
    du[3] = 0
end

nlls! (generic function with 1 method)

# Learnings and Notes

The Julia version at coding.csel.io is outdated and won’t be able to run the package or test cases

Install Julia locally and run: 
1. Using Pkg
2. Pkg.add("NonlinearSolve.jl")

Implements the following rootfinding 
- Newton Raphson, Trust Region, Levenberg Marquardt, Bisection, Falsi, etc. (full list found at links below)
    - https://docs.sciml.ai/NonlinearSolve/stable/solvers/nonlinear_system_solvers/
    - https://docs.sciml.ai/NonlinearSolve/stable/solvers/bracketing_solvers/


# Questions
- Why does this package implement so many different rootfinding methods?
    - When is it best to use each of them and why?
- What does each method require?
    - What causes different methods to fail?
    - Ex. bisection needs a positive and negative value in the interval

# Experiment Ideas
- Pick a subset of the methods included in NonlinearSolve.jl and evaluate the following
    - Performance (Accuracy, efficiency, etc.)
    - Conditions for each method
    - Impact of initial guess
    - Plot convergence of methods
- The documentation differentiates between the following types of problems: nonlinear system, interval rootfinding, steady state, non linear least squares
    - I think this exploration would be most interested in exploring methods for solving nonlinear system and interval rootfinding problems
        - Find subset within these two categories


# Running test suite

In [7]:
using Pkg
Pkg.add("ReTestItems")

   Resolving package versions...
     Project No packages added to or removed from `~/.julia/environments/v1.12/Project.toml`
    Manifest No packages added to or removed from `~/.julia/environments/v1.12/Manifest.toml`


In [8]:
using ReTestItems
runtests("./NonlinearSolve.jl/test/core_tests.jl"; name="NLLS Analytic Jacobian")

  Activating project at `~/Downloads/CSCI3656/NonlinearSolve.jl`
┌ Warning: Could not use exact versions of packages in manifest, re-resolving
└ @ TestEnv ~/.julia/packages/TestEnv/nGMfF/src/julia-1.11/activate_set.jl:76
[ Info: Scanning for test items in project `NonlinearSolve` at paths: ./NonlinearSolve.jl/test/core_tests.jl


00:38:28 | START (1/1) test item "NLLS Analytic Jacobian" at test/core_tests.jl:1
00:38:34 | DONE  (1/1) test item "NLLS Analytic Jacobian" 6.3 secs (99.7% compile, 1.6% recompile, 2.1% GC), 26.24 M allocs (1.471 GB)


[ Info: Finished scanning for test items in 0.96 seconds.
[ Info: Scheduling 1 tests on pid 51337


[ Tests Completed: 1/1 test items were run.
Test Summary:                | Pass  Total  Time
NonlinearSolve               |    1      1  7.0s
  test                       |    1      1      
    test/core_tests.jl       |    1      1      
      NLLS Analytic Jacobian |    1      1  6.3s


# Resources/Documentation
- Repository: https://github.com/SciML/NonlinearSolve.jl
- Documentation: https://docs.sciml.ai/NonlinearSolve/stable/